In [2]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime
import time
import plotly.graph_objects as go

def get_historical_data(symbol, timeframe, start_date, end_date):
    if not mt5.initialize():
        print("initialize() failed ☢️")
        mt5.shutdown()
        return None

    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    print(f"Retrieved {len(rates)} data points")

    if rates is None:
        print("Failed to retrieve historical data. ☢️")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    mt5.shutdown()

    return df


def calculate_rsi(df, period=14):
    try:
        rsi_indicator = momentum.RSIIndicator(df["close"], window=period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print(f"Error calculating RSI: {e}")

def find_filling_mode(symbol):
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_check(request)

        if result.comment == "Done":
            print("Trade was closed")
            break

    return i


In [3]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for sell conditions
        if current_bar["rsi"] > 70 and confirmation_bar["close"] > confirmation_bar["open"] \
                and previous_bar["close"] < previous_bar["open"] \
                and current_bar["close"] < current_bar["open"]:
            
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] < current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["entry_price"] - trades[-1]["exit_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades

"""
def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'])])

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        fig.add_trace(go.Scatter(x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='markers',
        marker=dict(size=[10, 10],
        symbol=['triangle-up', 'triangle-down'],
        color=['green', 'red'],
        line=dict(width=2))))

    fig.update_layout(title="Backtest Trades",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False)
    
    fig.show()
"""    

def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'],
                                        increasing_line_color='orange',
                                        decreasing_line_color='green',
                                        line_width=1)])

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        fig.add_trace(go.Scatter(x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='markers',
        marker=dict(size=[10, 10],
        symbol=['triangle-up', 'triangle-down'],
        color=['orange', 'green'],
        line=dict(width=2))))

    fig.update_layout(
        title="Backtest Trades",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=False,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,  # Remove x-axis gridlines
        yaxis_showgrid=False,  # Remove y-axis gridlines
    )

    fig.update_layout(width=800, height=800)

    fig.show()


if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 14

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 8, 9)
    end_date = datetime.now()

    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the strategy
    trades = backtest_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
        print(f"Profit/Loss: {trade['pnl']:.2f}\n")
        total_pnl += trade['pnl']

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(trades)}")

    # Step 5: Plot the trades
    plot_trades(df, trades)


Retrieved 986 data points
Trade Type: SELL
Entry Time: 2023-08-09 02:41:00, Entry Price: 11261.629
Exit Time: 2023-08-09 02:41:00, Exit Price: 11260.934
Profit/Loss: 0.70

Trade Type: SELL
Entry Time: 2023-08-09 03:37:00, Entry Price: 11270.686
Exit Time: 2023-08-09 03:37:00, Exit Price: 11270.03
Profit/Loss: 0.66

Trade Type: SELL
Entry Time: 2023-08-09 05:25:00, Entry Price: 11259.9
Exit Time: 2023-08-09 05:25:00, Exit Price: 11259.244
Profit/Loss: 0.66

Trade Type: SELL
Entry Time: 2023-08-09 07:24:00, Entry Price: 11241.507
Exit Time: 2023-08-09 07:24:00, Exit Price: 11240.89
Profit/Loss: 0.62

Trade Type: SELL
Entry Time: 2023-08-09 11:39:00, Entry Price: 11159.6
Exit Time: 2023-08-09 11:39:00, Exit Price: 11158.949
Profit/Loss: 0.65

Trade Type: SELL
Entry Time: 2023-08-09 14:54:00, Entry Price: 11089.125
Exit Time: 2023-08-09 14:54:00, Exit Price: 11088.448
Profit/Loss: 0.68

Total Profit/Loss: 3.95
Total Trades: 6


In [6]:
from plotly.subplots import make_subplots

def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'],
                                        increasing_line_color='orange',
                                        decreasing_line_color='green',
                                        line_width=1)])

    entry_exit_traces = []  # Store entry and exit traces separately

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        entry_trace = go.Scatter(x=[entry_time], y=[entry_price],
                                 mode='markers',
                                 marker=dict(size=10, symbol='triangle-up', color='orange'))
        
        exit_trace = go.Scatter(x=[exit_time], y=[exit_price],
                                mode='markers',
                                marker=dict(size=10, symbol='triangle-down', color='green'))

        entry_exit_traces.append(entry_trace)
        entry_exit_traces.append(exit_trace)

    # Calculate RSI levels
    rsi_levels = df['rsi']
    rsi_oversold = [30] * len(df)
    rsi_overbought = [70] * len(df)

    # Create a subplot with shared x-axis
    subplot_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Add candlestick chart to the first subplot
    subplot_fig.add_trace(go.Candlestick(x=df.index,
                                          open=df['open'],
                                          high=df['high'],
                                          low=df['low'],
                                          close=df['close'],
                                          increasing_line_color='orange',
                                          decreasing_line_color='green',
                                          line_width=1), row=1, col=1)

    # Add entry and exit traces to the first subplot
    for trace in entry_exit_traces:
        subplot_fig.add_trace(trace, row=1, col=1)

    # Add RSI plot to the second subplot
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_levels, name='RSI', line=dict(color='blue')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_oversold, name='Oversold', line=dict(color='red', dash='dash')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_overbought, name='Overbought', line=dict(color='green', dash='dash')), row=2, col=1)

    # Update layout of the subplots
    subplot_fig.update_layout(
        title="Backtest Trades with RSI",
        xaxis_title="Time",
        height=800,  # Set the height of the entire plot
        xaxis_rangeslider_visible=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
    )

    subplot_fig.show()


if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 8, 9)
    end_date = datetime.now()

    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the strategy
    trades = backtest_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
        print(f"Profit/Loss: {trade['pnl']:.2f}\n")
        total_pnl += trade['pnl']

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(trades)}")

    # Step 5: Plot the trades
    plot_trades(df, trades)


Retrieved 988 data points
Trade Type: SELL
Entry Time: 2023-08-08 23:10:00, Entry Price: 11298.135
Exit Time: 2023-08-08 23:10:00, Exit Price: 11297.549
Profit/Loss: 0.59

Trade Type: SELL
Entry Time: 2023-08-09 01:13:00, Entry Price: 11267.117
Exit Time: 2023-08-09 01:13:00, Exit Price: 11266.534
Profit/Loss: 0.58

Trade Type: SELL
Entry Time: 2023-08-09 02:16:00, Entry Price: 11248.083
Exit Time: 2023-08-09 02:16:00, Exit Price: 11247.5
Profit/Loss: 0.58

Trade Type: SELL
Entry Time: 2023-08-09 02:41:00, Entry Price: 11261.629
Exit Time: 2023-08-09 02:41:00, Exit Price: 11260.934
Profit/Loss: 0.70

Trade Type: SELL
Entry Time: 2023-08-09 02:48:00, Entry Price: 11261.614
Exit Time: 2023-08-09 02:48:00, Exit Price: 11261.01
Profit/Loss: 0.60

Trade Type: SELL
Entry Time: 2023-08-09 03:30:00, Entry Price: 11262.432
Exit Time: 2023-08-09 03:30:00, Exit Price: 11261.776
Profit/Loss: 0.66

Trade Type: SELL
Entry Time: 2023-08-09 03:37:00, Entry Price: 11270.686
Exit Time: 2023-08-09 03:37: